# Mesa Schelling example - A Schelling-like Segregation Model

## Description

The Schelling (1971) segregation model is a classic of agent-based modeling, demonstrating how agents following simple rules lead to the emergence of qualitatively different macro-level outcomes. Agents are randomly placed on a grid. There are two types of agents, one constituting the majority and the other the minority. All agents want a certain number (generally, 3) of their 8 surrounding neighbors to be of the same type in order for them to be happy. Unhappy agents will move to a more happier grid space (if available). While individual agents do not have a preference for a segregated outcome (e.g. they would be happy with 3 similar neighbors and 5 different ones), the aggregate outcome can nevertheless become heavily segregated.

## Sample Model Description

The tutorial model is a very simple simulated dynamic of agent segregation. These are the rules of our tutorial model:

1. There are some number of agents.
2. All agents begin as randomly placed on a grid.
3. At every step of the model, an agent considers how many of its surrounding neighbors to be of the same type in order for them to be happy.
4. If an agent is unhappy, it moves to an unoccupied cell; otherwise, an agent doesn't move

Despite its simplicity, this model yields results that are often unexpected to those not familiar with it.

## How to use and modify the code

These excercises are designed around a Mesa template that is given to you to reuse. You are not asked to perform any complex object programming, but instead we ask for understanding of the core features of the Mesa python package. If you want to add functionality or adapt certain features, you will mostly have to modifly the existing template and code the ``Model`` and ``Agent``behavior using standard python code.

Let’s get started.

# 1. Create the Basic Agent/Model

## Setting up the model

To begin writing the model code, we start with two core classes: one for
the overall model, the other for the agents. The model class holds the
model-level attributes, manages the agents, and generally handles the
global level of our model. Each instantiation of the model class will be
a specific model run. Each model will contain multiple agents, all of
which are instantiations of the agent class. Both the model and agent
classes are child classes of Mesa’s generic ``Model`` and ``Agent``
classes.

### ``# Agent Initialization``:

Each agent has only two variables:
- 2D position on the grid (x , y)
- agent type [0, 1]

(Each agent will also have a unique identifier (i.e., a position), stored in
the ``pos`` variable. Giving each agent a unique id is a good
practice when doing agent-based modeling.)

### ``# Calculate the number of similar neighbours``:

- calculate the number of similar neighbours
- if the Agent is happy with it's neighbourhood - it stays

### ``# Move to an empty location if unhappy``:

- if the Agent is unhappy - perform move to better empty cell
- if at new location there are no neighbors - agent moves

An example of the agent class can look like this:

In [ ]:
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid

In [ ]:
class SchellingAgent(Agent):
    """
    Schelling segregation agent
    """

    def __init__(self, pos, model, agent_type):
        """
        Create a new Schelling agent.

        Args:
           unique_id: Unique identifier for the agent.
           pos: Agent initial location.
           agent_type: Indicator for the agent's type (minority=1, majority=0)
        """
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type

    def step(self):
        similar = 0
        for neighbor in self.model.grid.iter_neighbors(self.pos, True):
            if neighbor.type == self.type:
                similar += 1

        # If unhappy, move:
        if similar < self.model.homophily:
            self.model.grid.move_to_empty(self)
        else:
            self.model.happy += 1

### ``# Model Initialization``:

There are a number of model-level parameters: 

- height and width of the grid
- density of grid population to define how many agents and empty cells the model contains
- minority proportion to define proportion of two types of agents on the grid
- homophily treshold to which the agent is happy with its neighbourhood

### ``# Create agents``:

- use uniform random numbers to populate the grid based on density parameter
- use uniform random numbers to selects agent type based on minority proportion

When a new model is started, we want it to populate the grid itself with the defined number of agents with having the given proportion between two groups. Let's implement an example model class:

In [ ]:
class Schelling(Model):
    """
    Model class for the Schelling segregation model.
    
    width: Horizontal axis of the grid which is used together with Height to define the total number of agents in the system.
    height: Vertical axis of the grid which is used together with Width to define the total number of agents in the system.
    density: Define the population density of agent in the system. Floating value from 0 to 1.
    fraction minority: The ratio between blue and red. Blue is represented as the minority while red is represented as the majority. Floating value from 0 to 1. If the value is higher than 0.5, blue will become the majority instead.
    homophily: Define the number of similar neighbors required for the agents to be happy. Integer value range from 0 to 8 since you can only be surrounded by 8 neighbors.
    """

    def __init__(self, width=20, height=20, density=0.8, minority_pc=0.2, homophily=3):
        super().__init__()
        self.width = width
        self.height = height
        self.homophily = homophily

        self.grid = mesa.space.SingleGrid(width, height, torus=True)

        self.happy = 0
        self.datacollector = mesa.DataCollector(
            {"happy": "happy"},  # Model-level count of happy agents
        )

        # Set up agents
        # We use a grid iterator that returns
        # the coordinates of a cell as well as
        # its contents. (coord_iter)
        for _, pos in self.grid.coord_iter():
            if self.random.random() < density:
                agent_type = 1 if self.random.random() < minority_pc else 0
                agent = SchellingAgent(pos, self, agent_type)
                self.grid.place_agent(agent, pos)

        self.datacollector.collect(self)

    def step(self):
        """
        Run one step of the model. If All agents are happy, halt the model.
        """
        self.happy = 0  # Reset counter of happy agents
        self.agents.shuffle().do("step")
        # Must be before data collection.
        self._advance_time()  # Temporary API; will be finalized by Mesa 3.0 release
        # collect data
        self.datacollector.collect(self)

        if self.happy == len(self.agents):
            self.running = False

# 2. Run the Agent/Model Basic

### Running the model

At this point, we have a model which runs.
You can see for yourself with a few easy lines. If you’ve been working
in an interactive session, you can create a model object directly. 

With that last piece in hand, it’s time for the first rudimentary run of
the model.

Now let’s create a model with 20 x 20 agents, and run it for 10 steps.

### ``# Model parameters``:

- specify all model-level parameters of its __init__ function
- height and width are given already as 20 x 20, as well as density and minority proportion
- define homophily threshold

In [ ]:
# Model parameters
model = Schelling(20, 20, 0.98, 0.5, 20)

while model.running:
    model.step()

# 3. Visualize the Agent/Model

### At the location ``# Model parameters``:

- specify all model-level parameters of its __init__ function
- height and width are given already as 20 x 20, as well as density and minority proportion
- given: homophily treshold 20%

In [ ]:
def get_happy_agents(model):
    """
    Display a text count of how many happy agents there are.
    """
    return f"Happy agents: {model.happy}"


def schelling_draw(agent):
    """
    Portrayal Method for canvas
    """
    if agent is None:
        return
    portrayal = {"Shape": "circle", "r": 0.5, "Filled": "true", "Layer": 0}

    if agent.type == 0:
        portrayal["Color"] = ["#FF0000", "#FF9999"]
        portrayal["stroke_color"] = "#00FF00"
    else:
        portrayal["Color"] = ["#0000FF", "#9999FF"]
        portrayal["stroke_color"] = "#000000"
    return portrayal


canvas_element = mesa.visualization.CanvasGrid(
    portrayal_method=schelling_draw,
    grid_width=20,
    grid_height=20,
    canvas_width=500,
    canvas_height=500,
)
happy_chart = mesa.visualization.ChartModule([{"Label": "happy", "Color": "Black"}])

model_params = {
    "height": 20,
    "width": 20,
    "density": mesa.visualization.Slider(
        name="Agent density", value=0.8, min_value=0.1, max_value=1.0, step=0.1
    ),
    "minority_pc": mesa.visualization.Slider(
        name="Fraction minority", value=0.2, min_value=0.00, max_value=1.0, step=0.05
    ),
    "homophily": mesa.visualization.Slider(
        name="Homophily", value=3, min_value=0, max_value=8, step=1
    )
}

server = mesa.visualization.ModularServer(
    model_cls=Schelling,
    visualization_elements=[canvas_element, get_happy_agents, happy_chart],
    name="Schelling Segregation Model",
    model_params=model_params,
)

# 4. Run the Agent/Model Visualization

NOTE: Runtime server error is normal and expected when running visualization code below. This visualization code was made for command line execution (not explicitly for Jupyter Notebooks), so we are **forcing** it's use.

Just make sure to increment the port number counter each visualization run, to be able to use it.

In [ ]:
server.port = 8523 # The default 8521 - increase the counter as you run the visalizations
server.launch() # Uncomment to run the visalization